In [1]:
import gc
import torch
import os

os.environ["TRANSFORMERS_CACHE"] = "/workspace/.cache/huggingface"
os.environ["HF_HOME"] = "/workspace/.cache/huggingface"
os.environ["TORCH_HOME"] = "/workspace/.cache/torch"
os.environ["XDG_CACHE_HOME"] = "/workspace/.cache"

gc.collect()
torch.cuda.empty_cache()

In [2]:
import os
from diffusers import StableDiffusionPipeline
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

token = os.getenv("HUGGINGFACE_TOKEN")
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"  # or try "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0", token=token)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

sd_model_id = "segmind/SSD-1B"
pipe2 = StableDiffusionPipeline.from_pretrained(
    sd_model_id,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
).to("cuda:1")

# # Optionally set auth token for private models
# pipe.enable_xformers_memory_efficient_attention()

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacty of 23.68 GiB of which 63.81 MiB is free. Process 3550723 has 23.62 GiB memory in use. Of the allocated memory 22.82 GiB is allocated by PyTorch, and 521.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:


user_input = "Generate a gym workout image prompt for training legs the prompt needs to be very extensive describe the action, the human and the surrounding as detailed as it could"
prompt = f"{user_input} in a professional cinematic lighting"

response = pipe(prompt, max_new_tokens=100)[0]["generated_text"]
print(response)
   
image = pipe2(response, num_inference_steps=30, guidance_scale=7.5).images[0]

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
image.save(f"output_{timestamp}.png")
# a
